priority 中能有额外数据的：地理位置，  经济实力，工作，专业，年龄，身高，学历
只有priority的单一数据的：三观，性格，星座，

##匹配大纲：##

#细节匹配#
地理位置匹配（50分）
偏好前三（有额外数据的，既能评分的）（10+10+10分）
偏好后三（5+5+5）

#偏好近似度#
作用：区分分数相同人群
（5分）

### 地理位置匹配（50分）

    User  exp_loc(20 score) 
        exp_hometown(20 score) 
        self_loc(5 score)
        self_home_town(5 score)
        大洲 国家 省 市 (各25%) hometown 为各 33，33，34分
   
### 偏好匹配（45分）（6种）（要求为空白皆0分，随缘0分）
#### 经济实力：
    除随缘要求，满足任意区间得满分
    如出现10w-20w，随缘；则删除随缘
#### 工作：
    除随缘要求，满足职位满分
#### 专业：
    除随缘要求，满足任意区间得满分，
    如出现理学，随缘；则删除随缘
    与我相同：查询该人专业并添加此要求
#### 年龄：
    在区间范围内满分
    在+- 2岁误差内75%分
    在+- 3岁误差内50%分
    在+- 5岁误差内25%分
    超过+-5cm 0分
#### 身高：
    在区间范围内满分
    在+- 2cm误差内75%分
    在+- 3cm误差内50%分
    在+- 5cm误差内25%分
    超过+-5cm 0分
#### 学历：
    除随缘要求，满足任意区间得满分，
    如出现本科，随缘；则删除随缘     

### 偏好近似度
    比较两者排名，选取前5偏好，计算共同偏好之间距离（待用）

In [5]:
import pandas as pd
import numpy as np

love_data = pd.read_excel('DeepMatchData.xlsx', sheet_name='Sheet1')
num_of_user = love_data.shape[0] 
love_data.head(5)
love_data

,uid,gender,age,tall,hometown_country,hometown_province,hometown_city,location_continent,location_country,location_province,...,XD_location_prority,XD_money_prority,XD_occupation_prority,XD_subject_prority,XD_values_prority,XD_age_prority,XD_appearance_prority,XD_character_prority,XD_constellation_prority,XD_degree_prority
0,1,女,21,NaN,中国,浙江省,嘉兴市,亚洲,新加坡,NaN,...,1.0,8.0,9.0,5.0,7.0,2.0,6.0,3.0,10.0,4.0
1,2,女,24,158.0,中国,广东省,深圳市,亚洲,中国,上海市,...,8.0,3.0,6.0,4.0,1.0,9.0,7.0,2.0,10.0,5.0
2,3,女,22,163.0,中国,重庆市,重庆市,亚洲,中国,北京市,...,9.0,8.0,7.0,5.0,3.0,6.0,1.0,4.0,10.0,2.0
3,4,女,27,168.0,中国,山东省,威海市,亚洲,中国,北京市,...,4.0,6.0,8.0,9.0,1.0,2.0,7.0,5.0,10.0,3.0
4,5,男,22,NaN,中国,河北省,唐山市,欧洲,芬兰,赫尔辛基,...,5.0,6.0,NaN,7.0,2.0,3.0,NaN,1.0,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,136,女,20,NaN,中国,山东省,青岛市,亚洲,中国,山东省,...,10.0,9.0,7.0,3.0,2.0,5.0,4.0,6.0,8.0,1.0
136,137,男,29,178.0,中国,陕西省,榆林市,亚洲,中国,陕西省,...,9.0,7.0,5.0,8.0,4.0,1.0,6.0,3.0,10.0,2.0
137,138,女,31,161.0,中国,四川省,乐山市,亚洲,中国,四川省,...,6.0,3.0,7.0,8.0,1.0,5.0,4.0,2.0,10.0,9.0
138,139,女,32,163.0,中国,安徽省,合肥市,欧洲,英国,英格兰,...,3.0,6.0,8.0,7.0,1.0,4.0,9.0,2.0,10.0,5.0


In [6]:
#区分男女，排除非异性恋

female_data = love_data[love_data['gender']=='女']
male_data = love_data[love_data['gender']=='男']

def detect_sex_pref(DataFrame):
    uidset = []
    uidset += DataFrame['uid'].where(DataFrame['gender']==DataFrame['XD_exp_gender2']).dropna().tolist() # 收集同性恋UID
    uidset += DataFrame['uid'].where(DataFrame['XD_exp_gender2']=='随缘').dropna().tolist() #收集双性恋UID
    return uidset

love_data1 = love_data[-love_data['uid'].isin(detect_sex_pref(love_data))]

female_data = love_data1[love_data1['gender']=='女']
male_data = love_data1[love_data1['gender']=='男']

# Data Description
print('After remove the bisexual and homosexual users with uid {}'.format(detect_sex_pref(love_data)))
print('We have {} female and {} male in this activity.'.format(female_data.shape[0],male_data.shape[0]))

female_data = female_data.set_index(['uid']) #把 UID 当作 index
male_data = male_data.set_index(['uid'])

non_heterosexual_list = detect_sex_pref(love_data)

After remove the bisexual and homosexual users with uid [77.0, 9.0, 44.0, 112.0]
We have 68 female and 68 male in this activity.


In [7]:
#地理位置评分

def location_matching(infor_A,infor_B): # list 长度为4
    if infor_A[3] == infor_B[3]:
        return 100
    elif infor_A[2] == infor_B[2]:
        return 75
    elif infor_A[1] == infor_B[1]:
        return 50
    elif infor_A[0] == infor_B[0]:
        return 25
    else:
        return 0

def home_town_matching(infor_A,infor_B): # list 长度为3
    if infor_A[2] == infor_B[2]:
        return 100
    elif infor_A[1] == infor_B[1]:
        return 66
    elif infor_A[0] == infor_B[0]:
        return 33
    else:
        return 0

def geo_score(requirement,sexual = 'male'): 
    # requirement 为 list，包含该用户的 
    # hometown_country hometown_province hometown_city 
    # location_continent location_country location_province location_city
    # XD_exp_continent XD_exp_country XD_exp_province XD_exp_city
    # XD_exp_hometown_country XD_exp_hometown_province XD_exp_hometown_city
    if sexual == 'male':
        DataFrame = female_data
    else:
        DataFrame = male_data
        
    scores_this_user = []
    for i in range(DataFrame.shape[0]):
        score_i = [] # 顺序：家乡得分 所在地的分 期望所在地 期望家乡
        infor_i = DataFrame.iloc[i][['hometown_country', 'hometown_province', 'hometown_city', 
                                    'location_continent', 'location_country', 'location_province', 'location_city']].tolist()
        
        score_i.append(home_town_matching(requirement[0:3], infor_i[0:3])) # Score for matching hometown
        score_i.append(location_matching(requirement[3:7], infor_i[3:7])) # Score for matching location
        score_i.append(location_matching(requirement[7:11], infor_i[3:7])) # Score for exp matching location
        score_i.append(home_town_matching(requirement[11:], infor_i[0:3])) # Score for exp matching hometown
#         print(score_i)
        scores_this_user.append(score_i)
    return np.array(scores_this_user)

In [8]:
# 6种偏好匹配
def age_tall_matching(infor_A, infor_B): # list A 长度为 2, B 为 1
    if np.isnan(infor_A[0]) and np.isnan(infor_A[1]):
        return 0 # 对年龄(身高)没有要求 得分 0
    elif np.isnan(infor_B):
        return 0
    elif ~np.isnan(infor_A[0]) and np.isnan(infor_A[1]): #只有上界
        if int(infor_B) <= int(infor_A[0]):
            return 100
        elif int(infor_B) <= int(infor_A[0]) + 2:
            return 75
        elif int(infor_B) <= int(infor_A[0]) + 3:
            return 50
        elif int(infor_B) <= int(infor_A[0]) + 5:
            return 25
        else:
            return 0
    elif np.isnan(infor_A[0]) and ~np.isnan(infor_A[1]): #只有下界
        if int(infor_B) >= int(infor_A[1]):
            return 100
        elif int(infor_B) >= int(infor_A[1]) - 2:
            return 75
        elif int(infor_B) >= int(infor_A[1]) - 3:
            return 50
        elif int(infor_B) >= int(infor_A[1]) - 5:
            return 25
        else:
            return 0
    else: #有双界
        if int(infor_B) <= int(infor_A[0]) and int(infor_B) >= int(infor_A[1]):
            return 100
        elif int(infor_B) <= int(infor_A[0]) + 2 and int(infor_B) >= int(infor_A[1]) - 2:
            return 75
        elif int(infor_B) <= int(infor_A[0]) + 3 and int(infor_B) >= int(infor_A[1]) - 3:
            return 50
        elif int(infor_B) <= int(infor_A[0]) + 5 and int(infor_B) >= int(infor_A[1]) - 5:
            return 25
        else:
            return 0
        

def degree_o_matching(infor_A,infor_B): # 合并了 学历 职业 计分方式
    if not isinstance(infor_A,str):
        return 0
    elif infor_B in infor_A.split(';'):
            return 100
    else:
        return 0
    
def subject_mathing(infor_A,infor_B,self_subject): ######## 注意 subject_level1 分类 和 exp_sub 并不相同 需改进
    if not isinstance(infor_A,str):
        return 0
    
    subject_list = infor_A.split(';')
    if '与我相同' in infor_A.split(';'):
        subject_list.append(self_subject) 
        
    if infor_B in subject_list:
        return 100
    else:
        return 0
    
def annual_income_matching(infor_A,infor_B):
    if not isinstance(infor_A,str):
        return 0
    if infor_B in infor_A.split(';'):
        return 100
    elif infor_B == '还没有工作':
        if '无工作的学生' in infor_A.split(';'):
            return 100
        else:
            return 0
    else:
        return 0

def six_pref_scores(requirement,sexual = 'male'):
    # requirement 为 list，包含该用户的 
    # XD_exp_age_max2 XD_exp_age_min2 XD_exp_tall_max XD_exp_tall_min
    # XD_exp_degree XD_exp_subject XD_exp_occupation XD_exp_annual_income
    # subject_level1
    if sexual == 'male':
        DataFrame = female_data
    else:
        DataFrame = male_data
        
    scores_this_user = []
    for i in range(DataFrame.shape[0]):
        score_i = [] # 顺序：年龄得分 身高得分 学历得分 专业得分 职业得分 收入得分
        infor_i = DataFrame.iloc[i][['age','tall','degree','subject_level1','occupation','annual_income']].tolist()
        
        score_i.append(age_tall_matching(requirement[0:2], infor_i[0])) # Score for age
        score_i.append(age_tall_matching(requirement[2:4], infor_i[1])) # Score for tall
        score_i.append(degree_o_matching(requirement[4], infor_i[2])) # Score for degree
        score_i.append(subject_mathing(requirement[5], infor_i[3], requirement[8])) # Score for subject
        score_i.append(degree_o_matching(requirement[6], infor_i[4])) # Score for occupation
        score_i.append(annual_income_matching(requirement[7], infor_i[5])) # Score for annual income

        scores_this_user.append(score_i)
    return np.array(scores_this_user)

In [9]:
pri_title = ['XD_location_prority','XD_money_prority','XD_occupation_prority','XD_subject_prority','XD_values_prority','XD_age_prority',
             'XD_appearance_prority','XD_character_prority','XD_constellation_prority','XD_degree_prority'
]

def priority_matching(infor_A, infor_B): # list 长度为 10
    A_idx_sorted = sorted(range(len(infor_A)), key=lambda k: infor_A[k])
    B_idx_sorted = sorted(range(len(infor_B)), key=lambda k: infor_B[k])
    return len([i for i in A_idx_sorted[0:5] if i in B_idx_sorted[0:5]])*20

def prio_scores(requirement,sexual = 'male'):
    # requirement 为 list，包含该用户的 
    # 'XD_location_prority','XD_money_prority','XD_occupation_prority','XD_subject_prority','XD_values_prority','XD_age_prority',
    # 'XD_appearance_prority','XD_character_prority','XD_constellation_prority','XD_degree_prority'
    
    if sexual == 'male':
        DataFrame = female_data
    else:
        DataFrame = male_data
        
    scores_this_user = []
    for i in range(DataFrame.shape[0]):
        score_i = [] 
        infor_i = DataFrame.iloc[i][pri_title].tolist()
        
        score_i.append(priority_matching(requirement, infor_i)) # Score for age

        scores_this_user.append(score_i)
    return np.array(scores_this_user)
            

In [10]:
geo_title = ['hometown_country', 'hometown_province', 'hometown_city', 
                                    'location_continent', 'location_country', 'location_province', 'location_city',
                                    'XD_exp_continent', 'XD_exp_country', 'XD_exp_province', 'XD_exp_city',
                                    'XD_exp_hometown_country', 'XD_exp_hometown_province', 'XD_exp_hometown_city']
six_title = ['XD_exp_age_max2', 'XD_exp_age_min2', 'XD_exp_tall_max', 'XD_exp_tall_min',
    'XD_exp_degree', 'XD_exp_subject', 'XD_exp_occupation', 'XD_exp_annual_income',
    'subject_level1']



def cal_score_for_i(uid): 
    if uid in male_data.index:
#         print('Calculate Scores for male, uid: {}'.format(uid))
        sexual = 'male'
        DataFrame = male_data
    elif uid in female_data.index:
#         print('Calculate Scores for female, uid: {}'.format(uid))
        sexual = 'female'
        DataFrame = female_data
    else:
        print('Can not find uid {}'.format(uid))
    
    req_geo = DataFrame.loc[uid][geo_title].tolist()
    req_six = DataFrame.loc[uid][six_title].tolist()
    req_prio = DataFrame.loc[uid][pri_title].tolist()
    
    cal_weight = DataFrame.loc[uid][pri_title].sort_values().drop(['XD_values_prority','XD_constellation_prority','XD_character_prority','XD_location_prority'])
    cal_weight[0:3] = 10
    cal_weight[3:] = 5
    six_weight = cal_weight[['XD_age_prority','XD_appearance_prority','XD_degree_prority','XD_subject_prority','XD_occupation_prority','XD_money_prority']].tolist()
    weight = [0.05,0.05,0.2,0.2] + [i*0.01 for i in six_weight] + [0.05]
#     print(weight)
    
    
    scores = np.hstack((geo_score(req_geo, sexual = sexual),six_pref_scores(req_six, sexual = sexual),prio_scores(req_prio, sexual = sexual)))
#     print(scores)
    weighted_scores = weight*scores
    return sexual, weight, scores, weighted_scores, np.sum(weighted_scores,axis = 1)

# OUTPUT COMPLETE SCORES

In [11]:
with open("Scores_for_Women.txt","w") as f:
        f.write("These scores is about men for every women, The second line is the score for frist women ans so on. Frist element in frist women line means the frist men's score to this frist women."+'\n')
        for i in female_data.index:
            Scores_txt = cal_score_for_i(i)[-1]
            Str = ''
            for j in range(68):
                Str = Str + str(round(Scores_txt[j], 2)) + ' '
            f.write(Str + '\n')

In [12]:
with open("Scores_for_Men.txt","w") as g:
        g.write("These scores is about women for every men, The second line is the score for frist men ans so on. Frist element in frist men line means the frist women's score to this frist men."+'\n')
        for i in male_data.index:
            Scores_txt = cal_score_for_i(i)[-1]
            Str = ''
            for j in range(68):
                Str = Str + str(round(Scores_txt[j], 2)) + ' '
            g.write(Str + '\n')

# OUTPUT REJECT LIST 

In [13]:
reject_data = pd.read_excel('DeepMatch活动数据历史匹配记录.xlsx', sheet_name='Sheet1')
reject_data

FileNotFoundError: [Errno 2] No such file or directory: 'DeepMatch活动数据历史匹配记录.xlsx'

In [ ]:
wrong_dating_purpose = []

for i in male_data.index:
    if male_data['relationship_status'].loc[i] == '想认真谈段恋爱，奔着结婚去':
        for j in female_data.index:
            if female_data['relationship_status'].loc[j] == '随便看看' or female_data['relationship_status'].loc[j] == '想谈一场不以结婚为目的的恋爱':
                wrong_dating_purpose.append([i,j])
    elif male_data['relationship_status'].loc[i] == '想谈一场不以结婚为目的的恋爱':
        for jj in female_data.index:
            if female_data['relationship_status'].loc[jj] == '想认真谈段恋爱，奔着结婚去':
                wrong_dating_purpose.append([i,jj])
        

for i in female_data.index:
    if female_data['relationship_status'].loc[i] == '想认真谈段恋爱，奔着结婚去':
        for j in male_data.index:
            if male_data['relationship_status'].loc[j] == '随便看看' or male_data['relationship_status'].loc[j] == '想谈一场不以结婚为目的的恋爱':
                wrong_dating_purpose.append([i,j])
    elif female_data['relationship_status'].loc[i] == '想谈一场不以结婚为目的的恋爱':
        for jj in male_data.index:
            if male_data['relationship_status'].loc[jj] == '想认真谈段恋爱，奔着结婚去':
                wrong_dating_purpose.append([i,jj])

wrong_dating_purpose

In [11]:
reject_list = [list(i) for i in list(reject_data.to_numpy())]

reject_list = reject_list + wrong_dating_purpose

# print(reject_list)
for i in range(len(reject_list)):
    reject_list[i] = sorted(reject_list[i])

# print(len(reject_list))    
    
no_repeat_reject_list = []

for i in reject_list:
    if i not in  no_repeat_reject_list:
        no_repeat_reject_list.append(i)
        

len(no_repeat_reject_list)


NameError: name 'reject_data' is not defined

In [12]:
First_Position = None

reject_pair = [] # number from 0 -

for i in no_repeat_reject_list:
    
    if i[0] in female_data.index and i[1] in male_data.index:
        First_Position = 'female'
    elif i[0] in male_data.index and i[1] in female_data.index:  
        First_Position = 'male'

    if i[0] not in non_heterosexual_list and i[1] not in non_heterosexual_list:
        if First_Position == 'female':
            female_number = female_data.index.to_list().index(i[0])
            male_number = male_data.index.to_list().index(i[1])
            reject_pair.append([female_number, male_number])
        elif First_Position == 'male':
            female_number = female_data.index.to_list().index(i[1])
            male_number = male_data.index.to_list().index(i[0])
            reject_pair.append([female_number, male_number])
            
    First_Position = None
    
with open("Reject_List.txt","w") as R:
    R.write('Each line indicates a unacceptable pairs. The first element is the index of women (from 0 to 68). The second one is the index of men (from 0 to 68)' + '\n')
    for i in reject_pair:
        R.write(str(i[0]) + ' ' + str(i[1]) + '\n')

NameError: name 'no_repeat_reject_list' is not defined